In [ ]:
import cv2
import numpy as np
import time
import os
import handdetectionmodule as hdm

In [ ]:
brushThickness=15
eraserThickness=90
drawColor=(255,0,255)

In [ ]:
folderpath="header"
myList=os.listdir(folderpath)
print(myList)
overlay=[]
for impath in myList:
    img=cv2.imread(f'{folderpath}/{impath}')
    overlay.append(img)
print(len(overlay))


In [ ]:
head=overlay[0]
cap=cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

In [ ]:
imgCanvas=np.zeros((720,1280,3),np.uint8)
detector=hdm.handDetector(detectionCon=0.85)
xp,yp=0,0

In [ ]:
while True:
    success,frame=cap.read()
    frame=cv2.flip(frame,1)

    #find hand landmark
    frame=detector.findHands(frame)
    list, bbox=detector.findPosition(frame,draw=False)

    if len(list)!=0:

        #print(list)

        #tip of index and middle finger
        x1, y1 = list[8][1:]
        x2, y2 = list[12][1:]

        fingers = detector.fingersUp()

        #If Selection Mode - Two finger are up
        if fingers[1] and fingers[2]:
            xp,yp=0,0
            #print("Selection Mode")

            if y1 < 125:
                if 250 < x1 < 450:        #purple brush
                    head = overlay[0]
                    drawColor = (255, 0, 255)
                elif 550 < x1 < 750:      #blue brush
                    head = overlay[1]
                    drawColor = (255, 0, 0)
                elif 800 < x1 < 1045:      #green brush
                    head = overlay[2]
                    drawColor = (0, 255, 0)
                elif 1050 < x1 < 1200:     #eraser
                    head = overlay[3]
                    drawColor = (0, 0, 0)
            cv2.rectangle(frame, (x1, y1 - 25), (x2, y2 + 25), drawColor, cv2.FILLED)     #selection mode is represented as rectangle
            #If Drawing Mode - Index finger is up
        if fingers[1] and fingers[2] == False:
            cv2.circle(frame, (x1, y1), 15, drawColor, cv2.FILLED)   #drawing mode is represented as circle
            #print("Drawing Mode")
            if xp == 0 and yp == 0:   #initially xp and yp will be at 0,0 so it will draw a line from 0,0 to whichever point our tip is at
                xp, yp = x1, y1    # so to avoid that we set xp=x1 and yp=y1

            #eraser
            if drawColor == (0, 0, 0):
                cv2.line(frame, (xp, yp), (x1, y1), drawColor, eraserThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, eraserThickness)
            else:
                cv2.line(frame, (xp, yp), (x1, y1), drawColor, brushThickness)#gonna draw lines from previous coodinates to new positions
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, brushThickness)

            xp,yp=x1,y1
    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)#on canvas all the region in which we drew is black and where it is black it is cosidered as white,it will create a mask
    imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)#converting again to gray bcoz we have to add in a RGB image i.e img
    frame = cv2.bitwise_and(frame,imgInv)
    frame = cv2.bitwise_or(frame,imgCanvas)

    frame[0:125, 0:1280]=head
    frame=cv2.addWeighted(frame,0.5,imgCanvas,0.5,0)
    cv2.imshow('Image', frame)
    #cv2.imshow('Canvas',imgCanvas)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
cap.release()
cv2.destroyAllWindows()